In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
path = "iris_flowers.csv"
try:
    df = pd.read_csv(path)
    print(df)
except FileNotFoundError:
    print("File not found")
except Exception as e:
    print("Error: {e}")

     5.1  3.5  1.4  0.2     Iris-setosa
0    4.9  3.0  1.4  0.2     Iris-setosa
1    4.7  3.2  1.3  0.2     Iris-setosa
2    4.6  3.1  1.5  0.2     Iris-setosa
3    5.0  3.6  1.4  0.2     Iris-setosa
4    5.4  3.9  1.7  0.4     Iris-setosa
..   ...  ...  ...  ...             ...
144  6.7  3.0  5.2  2.3  Iris-virginica
145  6.3  2.5  5.0  1.9  Iris-virginica
146  6.5  3.0  5.2  2.0  Iris-virginica
147  6.2  3.4  5.4  2.3  Iris-virginica
148  5.9  3.0  5.1  1.8  Iris-virginica

[149 rows x 5 columns]


In [3]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

In [4]:
df = pd.read_csv(path, header = None, names = column_names)
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
import numpy as np
label = 'species'
X = df.drop(label, axis = 1)
y2 = df['species'].values

In [7]:
print("Features shape:", X.shape)
print("Labels shape:", y2.shape)

Features shape: (150, 4)
Labels shape: (150,)


### Why do we encode?
ML models only understand numerical values
* OneHotEncoder will create separate columns for each category (only 1s and 0s, if true -> 1, if false -> 0)

In [8]:
encoder = OneHotEncoder(sparse_output = False)
y2_reshape = y2.reshape(-1,1)
y = encoder.fit_transform(y2_reshape)
print("Labels shape:", y.shape)

Labels shape: (150, 3)


In [9]:
y[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [10]:
encoder.categories_

[array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)]

### Why do we Scale?
To put all the features at the same playing field, to let the model learn more effectively.

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Features shape:", X_scaled.shape)

Features shape: (150, 4)


In [12]:
X_scaled[:5]

array([[-0.90068117,  1.03205722, -1.3412724 , -1.31297673],
       [-1.14301691, -0.1249576 , -1.3412724 , -1.31297673],
       [-1.38535265,  0.33784833, -1.39813811, -1.31297673],
       [-1.50652052,  0.10644536, -1.2844067 , -1.31297673],
       [-1.02184904,  1.26346019, -1.3412724 , -1.31297673]])

### Splitting the data

In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size = 0.2, random_state = 42, stratify = y
)

In [14]:
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size = 0.5, random_state = 42, stratify = y_temp
)

### Training the Model

In [15]:
input_shape = (X_train.shape[1],)
model = Sequential([
    Dense(8, activation = 'relu', input_shape = input_shape),
    Dense(4, activation = 'relu'),
    Dense(3, activation = 'softmax')
])
model.summary()

C:\Users\BEDANTA\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │              15 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 91 (364.00 B)

 Trainable params: 91 (364.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer = Adam(learning_rate = 0.1),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [17]:
history = model.fit(
    X_train, y_train,
    validation_data = (X_val, y_val),
    epochs = 100,
    batch_size = 32,
    verbose = 1
)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.3250 - loss: 1.1467 - val_accuracy: 0.4667 - val_loss: 0.9111
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6250 - loss: 0.7556 - val_accuracy: 0.6667 - val_loss: 0.6357
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6667 - loss: 0.6015 - val_accuracy: 0.6667 - val_loss: 0.5098
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6750 - loss: 0.4904 - val_accuracy: 0.9333 - val_loss: 0.3681
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9417 - loss: 0.3560 - val_accuracy: 1.0000 - val_loss: 0.2545
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9667 - loss: 0.2364 - val_accuracy: 1.0000 - val_loss: 0.1784
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9500 - loss: 0.1857 - val_accuracy: 0.9333 - val_loss: 0.1927
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9500 - loss: 0.1725 - val_accuracy: 0.9333 - val_loss

In [18]:
loss, accuracy = model.evaluate(X_test, y_test, verbose = 1)
print(f"Test Set Loss:     {loss:.4f}")
print(f"Test Set Accuracy: {accuracy:.4f} (or {accuracy*100:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9333 - loss: 0.1075
Test Set Loss:     0.1075
Test Set Accuracy: 0.9333 (or 93.33%)
